# Personalization Project 1 Report


**Jessie Du** (yd2464, @chopperase), **Claire Feng** (yf2476, @claireyyf), **Zijun Huang** (zh2356, @zijunh19), **Weisi Yan** (wy2320, @weisiyan)


# 1. Introduction 

The project aims to build a recommendation system for movies based on historical user ratings. We assume that the digital media company we work at owns a website that aggregates all kinds of information related to movies. The goal of the system is to **provide each user a list of personalized movie recommendations** as a mean to enhance user experience, increase customer loyalty, and thus improve the company bottomeline/profit. With a good recommendation system, the company would also be able to attract more users and rely on existing ones to share information on the website. 

Our recommendation system is **user-based** because we  want to create a sense of community on the platform by recommending movies to users based on people with similar taste (‘people like you also watched…’). 

# 2. Dataset Overview

The data used for this project is the 'ratings.csv' file from the 'MovieLens 20M Dataset', which could be accessed here: <a href='https://grouplens.org/datasets/movielens/'>https://grouplens.org/datasets/movielens/</a>. The ratings are recorded as per user-per movie. The columns are: userId, movieId, ratings (on a scale of 1 to 5), and timestamp (UTC since 1970/1/1). The original data contains ratings of 138,493 users on 26,744 movies from 1995 to 2015. Before sampling for our models, we did some high-level cleaning:

* **Recent 5 years (2010-2015)**: set 5-year as a window during which we assume people’s taste in movie does not change much. This range also guarantees a relatively wide range of choices for our models. 

* **Movies with at least 20 ratings over the past 5 years**: to ensure data density so that movies included in the sample still have enough ratings to be split into training and testing set. Also, given the size of the original dataset (and the fact that some movies have been rated 10,000+ times), if one movie is rated for less than 20 times over the five-year period, there is a high possibility that the movie is either too niche or too old - or too bad, all of which make them less than ideal to be recommended to our users.  

* **Users with at least 20 ratings over the past 5 years**: to stay consistent with the original dataset, in which each user rated at least 20 movies. This filter also serves to ensure that we are only referencing users who are during the five-year timeframe. 


# 3. Neighborhood-based model:

## 3.1 Model setup 

In addition to the parameters above, we further reduced the dimensionality of sample data to expedite model runtime by keeping 30% of movies rated by each user. We tried different parameters - 100%, 50%, 30%, and 10%. As we went down the percentage, there was not much change in the distribution of data over the first three quantiles, but the variance kept decreasing. 30% was chosen as the final parameter because it was the smallest percentage that still caused an effective shrinkage in both data size and variance. 

In the given sample dataset, we already have user-movie pairs with ratings, so memory-based collaborative filtering model is a straightforward approach to building the recommendation system. In this project, we decided on the **user-based**  method. We treat users as vectors and calculate the similarity between pairs of users. Due to the large number of movies and small number of ratings for each user, we have a *sparse matrix* to store the rating information. The graph below shows that the number of ratings are very unevenly distributed.

<img src = 'images_KNN/sample_1000.png' width='500' height='250' pos='center'>

To calculate the user-user similarity, we used *cosine function*. We assume that if user $u$ and user $v$ have rated the same movies $j$, we regard $(u,j)$ and $(v,j)$ as the peer set and store its rating. If one of them did not rate $j$, then we dropped the record. After we found all peer-set ratings of user $u$ and user $v$, we calculated the similarity score between them by using the peer-set ratings. After we got the user-user similarity, for each user, we found the K closest neighbors to user $u$ and stored them. Here, **K** is a hyperparameter.


$$Sim(user_{a},user_{b}) = cos(\theta ) = \frac{a\cdot b}{{\left \|a  \right \|}\left \| b \right \|}$$

Based on the similarity score for user, the predicted rating for user $u$ to movie $j$ is calculated using the following formula:

<img src='images_KNN/prediction.png' width='400' height='200' pos='center'>

Since we already have the predicted ratings, we need to find the *accuracy* of the model, which will be explained in the following parts. For each user in the train set, we would recommend *top-N movies they would like* based on the predicted ratings. We also found movies they watched in the test set to do the comparison and get *precision, recall and coverage*. Besides that, we ran code to find baseline accuracy of RMSE = 0.7178. 

## 3.2 Model evaluation
To evaluate the model, we split the whole dataset into train (70%) and test (30%). In the analysis part, we sample the data from 100 unique users to 974 an finally increase to 4592 to do the analysis.

<img src='images_KNN/train_test.png' width='300' height='150' pos='center'>

The **Error accuracy** we used here is *RMSE*. We calculate it based on the rating user rated in the train set and predicted ratings for the rated movies. With the enlarge of the datasize, we can see the change of RMSE. In the Baseline, we found RMSE for train set is 0.7178. In our model, we can with the user size increase from 100 to 1000, the RMSE decrease a lot from 1.04 to 0.88, then, if we still enlarge our data set, the RMSE changes smoothly. As we can see, the RMSE is around 0.86 which is large that means our model can be improved. 

<img src ='images_KNN/RMSE_change.png' width='400' height='200' pos='center'>

The **Ranking accuracy** we used here is precision and recall. We used 'R' as the set of N recommended movies for a user u and 'W' be the true movies user u watched. 'I' is the total number of movies in the test set:

Precision: $$ P_u=\frac{\left \|R(t)\bigcap W\right \|}{R(t)}$$

Recall: $$R_u=\frac{\left \|R(t)\bigcap W\right \|}{W}$$

Coverage:  $$C_u=\frac{\left \| R(t) \right \|}{I}$$

As we can see, as the number of user increases, model performance does not improve much, so the overall accuracy remains the same. However, the running time increases with the sample size. 

<img src ='images_KNN/time_used.png' width='400' height='200' pos='center'>

## 3.3 Model exploration

For hyper parameters in this algorithms, we have the **Top K closest neighbours** to the user, the **Top N movies** recommended to user and the **sample size**. 

To find the best K, we use RMSE of train set to test the performance of the model, we can see that with the increase of K, the RMSE decrease smoothly from 1.04 to around 0.8. Because for loop take a long time to run, we only graph k from 1 to 20 as below. We can see that with. increase of K, the RMSE decreases slowly and we use 15 closest neighbours to calculate the user-user similarity. In fact, in our testing, we found when K is 51 we have the smallest RMSE, but due to the slow decreases in RMSE, we choose **15 as the best_k**. After applying 15 in the model, when the we use our dataset, the RMSE for train set is 0.888 and test set is 0.919. If we use best k = 51 to test, the RMSE for train is 0.812 and 0.791 for test. In baseline, the RMSE is 0.7834.

<img src = 'images_KNN/best_k.png' width='400' height='200' pos='center'>

With the fixed K, we set N to be 10 at first, we found the precision is 0.0248, recall is 0.0291, coverage is 0.305.

We tune N by set the $N = [5,10,15,20,25,30,35,40]$ and get the changes of precision, recall and coverage. The more movie we recommend to users, the higher the coverage becomes and may even exceed 1. But precision shows a differnt trend. The intersection point of precision and recall line is at **N=7**, which could be used as the best N. When N is 7, the coverage is 0.28 which means our model do not have a good ability to cover all itmes.

<img src = 'images_KNN/precision.png' width='400' height='200' pos='center'>

## 3.4 Business value
Regardless of a user's level of activeness on rating movies (as long as the user is active), we would be able to recommend the user *new* movies, based on similar users' activities. Even though we set the number of recommendation to 7, it could be easily scale to a larger number without much sacrifice in accuracy metrics. All of these considerations boil down to a trade-off between accuracy and abundance. With a large N, we could achieve very high coverage (e.g. 70%), but users might only click very few of the recommended movies. Yet, iff more new movies are recommended to users, they might discover something new and unexpected as well. 

## 3.5 Potential Improvements
* For uesr-based model, the way we sample the data may significantly affect the results. The more movies the user have rated, the higher accuracy we have. User vectors are very sparse in our dataset, so we could change to *item vectors*, which have been proven provide better results. 
* We could change the way we calculate similarity. Inner product, cosine, pearson and jaccard, etc have extremely different performance in the diverse rating types.
* In the prediction function, we could use *z-score method* to do the recommendation. 
* When choosing the best parameters: N and K, we had to make a subjective call. The value of K influences the performance of precision, recall and coverage. So the way to find a best K and N to improve the accuracy of model need take further steps to find. 

A final concern is that memory-based model in general is not very practical. Runtime increases quite sharply with data size. And in reality, recommendations should ideally be done in real-time (e.g. once a user provides a rating, we should update the recommendations). So long runtime would hurt the company as more users come onboard to rate movies.

# 4. Matrix Factorization

## 4.1 Model setup

From the current dataset, we randomly selected 5,000 users and 500 movies and use the corresponding data as our sample - we came up with a sample of 4536 users and 496 movies. We used the SVD algorithm from scipy.sparse.linalg package to implement the model and took 50 as default value for latent dimension. 

Our recommender selected the top 10 movies that have the highest predicted ratings and are not rated in the train set to recommend to each user. 


## 4.2 Evaluation methods
We implemented a 5-fold cross validation. 

We first calculated RMSE to assess model performance. However, RMSE doesn’t necessarily tell us how relevant our recommended movies to each user is. Since our business objective is to recommend movies that are relevant and likely to raise users’ interest to explore further, we decided to use precision and recall as our primary accuracy metrics and coverage as secondary metric.


## 4.3 Model exploration (hyperparameter & sample size)
To tune the hyperparameter, we evaluate performance of the model taking different values [1,2,3,4,5] for latent dimension. From the plots of precisions and recalls, we see that the optimal value for latent dimension is 1 where the model achieves 6.64% precision, 34.1% recall, 7.51% coverage on the train set. We re-train this model on the entire training data and assess its performance on the test set: 10.32% precision, 34.23% recall, 7.22% coverage. 

    size=5000 users, 500 movies
<img src ='images/svd_precision.png'>   <img src ='images/svd_recall.png'>

Then, we implement the SVD model on a larger sample. Now we randomly select 10,000 users and 1,000 movies and end up with a sample of 9,855 users and 1,000 movies. We tune the hyperparameter again and find the optimal number of latent dimensions is still 1. The model has 6.38% precision, 22.4% recall, 3.27% coverage on the train set and has 11.5% precision, 23.13% recall, 3.75% coverage on the test set. 

    size=10000 users, 1000 movies
       
<img src ='images/svd_large_precision.png'>   <img src ='images/svd_large_recall.png'>

We can see that overall precision improves while recall and coverage decrease with a larger sample size. For the original small sample, it took 6 minutes to finish the model selection process (hyperparameter tuning); for the larger sample, it took 24 minutes. 


## 4.4 Business value

Our model mainly generates recommendations from very popular movies. Although such recommendations might bore some users, well, these movies are popular because they are liked by most people. Most users would be curious to know the cast, ratings, reviews, etc., or they might leave a rating or review of their own - now we successfully attract them to stay longer on the website and collect more data! 


## 4.5 Potential Improvements

In collaborative filtering, the more ratings data there are available for each user and for each item, the more personalized can the generated recommendations be. However, in our sample, first of all, many users have a small number of ratings; second, popular movies have significantly more ratings than the others. For example, 75% of the movies in our sample have less than 54 ratings but the popular movies have hundreds of ratings. Therefore, the model tends to recommend only from the popular group and the coverage is rather small. 



# 5. Conclusion

## 5.1 Business objectives

The goal of the recommendation systems built in this project is to increase the company profit **providing each user a list of personalized new movie recommendations**. This feature serves to enhance user experience, increase customer loyalty, and attract new users. With a good recommendation system, the company would also be able to attract more users and rely on existing ones to share information on the website. 

## 5.2 Future improvement

As for the next steps for improvement for collaborative filtering model, we could try item-beasd to do a deeper analyzing. 

As for the next steps for improvement for matrix factorization model, we could try other algorithms besides SVD, such as ALS. Also, we could use distributed computing framework like SparkML to implement the model and increase the speed of factorization calculation. 

In the current model, we only look at users and movies, while there are other factors that haven’t been taken into consideration, such as genres and tags. Using additional information in the recommendation system can help further personalize the experience for users.  